# Instalação de bibliotecas

In [ ]:
!pip install tensorflow==2.6.0
!pip install keras==2.6.0
!pip install numpy==1.19.5 pandas
!pip install protobuf==3.19.0
!pip install scipy==1.7.3
!pip install motmetrics==1.4.0


#Importa bibliotecas

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd  # Importa o Pandas para manipulação e análise de dados em DataFrames
import tensorflow as tf  # Importa o TensorFlow para construção e treinamento de modelos de machine learning
from tensorflow.keras.layers import Input, Dense, Dropout, GRU, LeakyReLU  # Importa camadas específicas da Keras para criação de redes neurais
from tensorflow.keras.models import Sequential  # Importa o modelo sequencial para criar modelos empilhando camadas
from tensorflow.keras.optimizers import Adam  # Importa o otimizador Adam para ajuste dos pesos da rede neural
from tensorflow.keras.callbacks import EarlyStopping  # Importa o callback EarlyStopping para interromper o treinamento se não houver melhoria
from sklearn.model_selection import train_test_split  # Importa função para dividir dados em conjuntos de treino e teste
from sklearn.preprocessing import MinMaxScaler  # Importa o MinMaxScaler para normalizar os dados em uma escala entre 0 e 1
import numpy as np  # Importa o NumPy para operações numéricas e manipulação de arrays
import random  # Importa random para gerar números aleatórios, útil para controle de aleatoriedade
import os  # Importa os para manipulação de arquivos e diretórios no sistema
import motmetrics as mm  # Importa motmetrics para cálculo de métricas de rastreamento de objetos
from scipy.spatial.distance import cdist  # Importa cdist para cálculo de distâncias entre arrays, útil em associações de rastreamento
import itertools  # Importa itertools para criação de iteradores complexos, usado em operações combinatórias


# Define semente e carrega dataset filtrado

In [ ]:
# Definir seed para reprodutibilidade
SEED = 42  # Define um valor de semente fixa para garantir a reprodutibilidade dos resultados
np.random.seed(SEED)  # Define a seed para as operações aleatórias do NumPy
tf.random.set_seed(SEED)  # Define a seed para as operações aleatórias do TensorFlow
random.seed(SEED)  # Define a seed para as operações aleatórias da biblioteca random
os.environ['PYTHONHASHSEED'] = '0'  # Define o hash do Python como constante para consistência nos resultados

# Carregar o arquivo CSV
path_filtered = 'https://drive.google.com/uc?id=1PBBHU8JvnOBBVeV0HcyLjcxyR8upZafX'  # Caminho do arquivo CSV armazenado no Google Drive
data = pd.read_csv(path_filtered)  # Carrega o arquivo CSV para um DataFrame usando o Pandas


# Preparação dos dados

In [ ]:
# Função para preparar os dados
def prepare_data_for_prediction(data, sequence_length, feature_cols, target_cols):
    # Ordenar o DataFrame por instance_token, camera e timestamp para garantir a sequência correta
    data = data.sort_values(by=['instance_token', 'camera', 'timestamp']).reset_index(drop=True)

    # One-hot encode da coluna 'camera'
    camera_dummies = pd.get_dummies(data['camera'], prefix='camera')  # Cria variáveis binárias para cada valor da coluna 'camera'
    data = pd.concat([data, camera_dummies], axis=1)  # Concatena as novas colunas ao DataFrame original

    # Atualizar feature_cols com as novas colunas
    feature_cols = feature_cols + list(camera_dummies.columns)  # Adiciona as colunas de 'camera' codificadas em 'feature_cols'

    # Criar sequências
    sequences = []  # Lista para armazenar sequências de características
    targets = []  # Lista para armazenar os alvos
    metadata_list = []  # Lista para armazenar metadados associados a cada sequência

    # Agrupar dados por instance_token e camera
    grouped = data.groupby(['instance_token', 'camera'])  # Agrupa o DataFrame com base nas colunas instance_token e camera

    for (instance_token, camera), group in grouped:
        group = group.reset_index(drop=True)  # Reseta o índice do grupo para facilitar o acesso sequencial
        num_samples = len(group)  # Calcula o número de amostras no grupo
        if num_samples <= sequence_length:
            continue  # Pula o grupo se não houver amostras suficientes para uma sequência completa
        for i in range(sequence_length, num_samples):
            seq_features = group.loc[i - sequence_length:i - 1, feature_cols].values  # Extrai as características da sequência
            target = group.loc[i, target_cols].values  # Define o alvo como a linha atual da sequência
            sequences.append(seq_features)  # Adiciona a sequência de características à lista de sequências
            targets.append(target)  # Adiciona o alvo à lista de alvos
            metadata_list.append({
                'instance_token': instance_token,
                'timestamp': group.loc[i, 'timestamp'],
                'camera': camera,
                'scene_token': group.loc[i, 'scene_token']
            })  # Adiciona o metadado associado à sequência atual

    # Converter listas para arrays numpy
    sequences = np.array(sequences)  # Converte a lista de sequências para um array NumPy
    targets = np.array(targets)  # Converte a lista de alvos para um array NumPy

    # Converter metadata_list em DataFrame
    metadata = pd.DataFrame(metadata_list)  # Converte a lista de metadados para um DataFrame para fácil acesso e manipulação

    # Garantir que os tipos dos dados sejam float32
    sequences = sequences.astype(np.float32)  # Define os tipos das sequências como float32 para consistência
    targets = targets.astype(np.float32)  # Define os tipos dos alvos como float32 para consistência

    # Verificar se há valores NaN
    if np.isnan(sequences).any() or np.isnan(targets).any():
        print("Dados contêm NaNs. Por favor, verifique seus dados.")  # Informa ao usuário que há valores NaN
        raise ValueError("Dados contêm NaNs.")  # Lança um erro caso haja valores NaN nos dados

    # Verificar consistência dos comprimentos
    assert len(sequences) == len(targets) == len(metadata), "Inconsistência nos comprimentos das listas de saída."  # Garante que todas as listas tenham o mesmo comprimento

    return sequences, targets, metadata  # Retorna as sequências, alvos e metadados


# Divide os dados em treino e teste 80% e 20% respectivamente

In [ ]:
# Dividir os dados em treino e teste garantindo que cenas não sejam misturadas
unique_scenes = data['scene_token'].unique()  # Obtém uma lista única de cenas presentes na coluna 'scene_token'
train_scenes, test_scenes = train_test_split(unique_scenes, test_size=0.2, random_state=SEED)  # Divide as cenas em treino e teste, usando 20% para teste e garantindo reprodutibilidade com a seed

train_data = data[data['scene_token'].isin(train_scenes)].reset_index(drop=True)  # Seleciona os dados de treino com base nas cenas de treino e reseta o índice
test_data = data[data['scene_token'].isin(test_scenes)].reset_index(drop=True)  # Seleciona os dados de teste com base nas cenas de teste e reseta o índice


# Define todos os Hiperparâmetros que serão testados

In [ ]:
# Hiperparâmetros a serem testados
sequence_lengths = [4, 5, 6, 7]  # Lista de comprimentos de sequência para testes, definindo o número de passos de tempo na entrada
units_list = [256, 32, 64, 128, 512]  # Lista de tamanhos de unidades nas camadas GRU, definindo o número de neurônios por camada
dropout_rates = [0.1, 0.2]  # Taxas de dropout para regularização, ajudando a evitar overfitting
epochs_list = [10, 30, 200]  # Quantidade de épocas para treinamento, definindo o número de vezes que o modelo verá os dados de treino
activations = ['swish', 'relu', 'linear', 'selu', LeakyReLU(), 'tanh']  # Lista de funções de ativação a serem testadas para as camadas ocultas
learning_rates = [0.001]  # Taxas de aprendizado para o otimizador Adam, controlando a velocidade de ajuste dos pesos

# Definir colunas de features e target
base_feature_cols = ['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']  # Colunas de características principais usadas como entrada no modelo
target_cols = ['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']  # Colunas de alvo que o modelo irá prever


# Aplicação do método RNN - GRU

In [ ]:
# Função para criar o modelo GRU Sequencial
def create_GRU_model(sequence_length, num_features, units, dropout_rate, activation, learning_rate):
    model = Sequential()  # Inicializa o modelo sequencial
    model.add(Input(shape=(sequence_length, num_features)))  # Adiciona a camada de entrada com o comprimento da sequência e número de features
    model.add(GRU(units, activation=activation))  # Adiciona uma camada GRU com o número de unidades e a função de ativação especificada
    model.add(Dropout(dropout_rate))  # Adiciona uma camada de Dropout para regularização
    model.add(Dense(4, activation='linear'))  # Adiciona uma camada densa para saída, prevendo [min_x, max_x, min_y, max_y]
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mae'])  # Compila o modelo com o otimizador Adam, função de perda e métrica de erro absoluto médio (MAE)
    return model  # Retorna o modelo configurado

# Função para calcular o IoU de forma vetorizada (forma mais eficiente)
def calcular_iou_vetorizado(bbox_real, bbox_predito):
    xA = np.maximum(bbox_real[:, 0], bbox_predito[:, 0])  # Calcula a coordenada x do canto superior esquerdo da interseção
    yA = np.maximum(bbox_real[:, 2], bbox_predito[:, 2])  # Calcula a coordenada y do canto superior esquerdo da interseção
    xB = np.minimum(bbox_real[:, 1], bbox_predito[:, 1])  # Calcula a coordenada x do canto inferior direito da interseção
    yB = np.minimum(bbox_real[:, 3], bbox_predito[:, 3])  # Calcula a coordenada y do canto inferior direito da interseção

    interArea = np.maximum(0, xB - xA) * np.maximum(0, yB - yA)  # Calcula a área da interseção, garantindo que seja não-negativa
    boxA_area = (bbox_real[:, 1] - bbox_real[:, 0]) * (bbox_real[:, 3] - bbox_real[:, 2])  # Calcula a área da bounding box real
    boxB_area = (bbox_predito[:, 1] - bbox_predito[:, 0]) * (bbox_predito[:, 3] - bbox_predito[:, 2])  # Calcula a área da bounding box predita

    epsilon = 1e-7  # Define uma pequena constante para evitar divisão por zero
    iou = interArea / (boxA_area + boxB_area - interArea + epsilon)  # Calcula o IoU usando a área de interseção e as áreas das bounding box
    return iou  # Retorna o valor do IoU

best_models = []  # Inicializa uma lista para armazenar os melhores modelos encontrados


# Itera todos os Hiperparâmetros entre si, NÃO escolhe aleatóriamente, testa todas as combinações.

In [ ]:
# Loop sobre cada combinação de hiperparâmetros
for (sequence_length, units, dropout_rate, learning_rate, activation, epochs) in itertools.product(sequence_lengths, units_list, dropout_rates, learning_rates, activations, epochs_list):
    activation_name = activation if isinstance(activation, str) else activation.__class__.__name__  # Obtém o nome da ativação, verificando se é string ou classe
    print(f'\nTreinando com hiperparâmetros: sequence_length={sequence_length}, units={units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, activation={activation_name}, epochs={epochs}')

    # Atualiza feature_cols
    feature_cols = base_feature_cols.copy()  # Faz uma cópia das colunas de características para serem usadas nesta configuração

    # Prepara os dados de treino e teste
    X_train, y_train, _ = prepare_data_for_prediction(train_data, sequence_length, feature_cols, target_cols)  # Prepara os dados de treino
    X_test, y_test, test_metadata = prepare_data_for_prediction(test_data, sequence_length, feature_cols, target_cols)  # Prepara os dados de teste

    # Verifica se há dados suficientes
    if X_train.shape[0] == 0 or X_test.shape[0] == 0:
        print("Dados insuficientes para o sequence_length especificado. Pulando esta combinação.")  # Verifica se há dados suficientes para esta sequência
        continue

    # Garanti que os tipos dos dados sejam float32
    X_train = X_train.astype(np.float32)  # Define X_train como float32 para consistência
    y_train = y_train.astype(np.float32)  # Define y_train como float32
    X_test = X_test.astype(np.float32)  # Define X_test como float32
    y_test = y_test.astype(np.float32)  # Define y_test como float32

    # Verifica se há valores NaN
    if np.isnan(X_train).any() or np.isnan(y_train).any() or np.isnan(X_test).any() or np.isnan(y_test).any():
        print("Dados contêm NaNs. Pulando esta combinação.")  # Se houver valores NaN, pula para a próxima combinação
        continue

    # Normaliza as coordenadas das bounding boxes
    scaler = MinMaxScaler()  # Inicializa o scaler para normalizar os dados entre 0 e 1
    num_features = X_train.shape[2]  # Número de características na entrada
    bbox_indices = [feature_cols.index(col) for col in target_cols]  # Índices das colunas de bounding boxes para normalizar

    # Flatten X_train para ajustar o scaler
    X_train_flat = X_train.reshape(-1, num_features)  # Achata X_train para ajustá-lo ao scaler

    # Combina as coordenadas de X_train e y_train para ajustar o scaler
    all_bbox_train = np.vstack([X_train_flat[:, bbox_indices], y_train])  # Junta coordenadas de X_train e y_train para ajustar o scaler de forma consistente
    scaler.fit(all_bbox_train)  # Ajusta o scaler aos dados combinados

    # Transforma X_train
    X_train_flat[:, bbox_indices] = scaler.transform(X_train_flat[:, bbox_indices])  # Aplica a normalização ao X_train nas coordenadas das bounding boxes
    X_train = X_train_flat.reshape(-1, sequence_length, num_features)  # Transforma X_train para seu formato original

    # Transforma y_train
    y_train = scaler.transform(y_train)  # Normaliza y_train usando o scaler ajustado

    # Transforma X_test
    X_test_flat = X_test.reshape(-1, num_features)  # Achata X_test para ajustar o scaler
    X_test_flat[:, bbox_indices] = scaler.transform(X_test_flat[:, bbox_indices])  # Aplica a normalização ao X_test
    X_test = X_test_flat.reshape(-1, sequence_length, num_features)  # Transforma X_test para seu formato original

    # Transformar y_test
    y_test = scaler.transform(y_test)  # Normaliza y_test

    # Constroe o modelo
    GRU_model = create_GRU_model(sequence_length, num_features, units, dropout_rate, activation, learning_rate)  # Cria o modelo GRU com os hiperparâmetros definidos

    # Implementar EarlyStopping - Interrompe o treinamento caso os resultados parem de melhorar
    early_stopping = EarlyStopping(
        monitor='val_loss',  # Monitora a perda de validação para determinar o ponto de parada
        patience=10,  # Define o número de épocas sem melhora para interromper o treinamento
        restore_best_weights=True  # Restaura os melhores pesos ao final do treinamento
    )

    # Treina o modelo
    GRU_model.fit(
        X_train, y_train,
        epochs=epochs,  # Número de épocas definido
        batch_size=32,  # Tamanho do lote de treinamento
        validation_data=(X_test, y_test),  # Dados de validação para avaliação durante o treinamento
        callbacks=[early_stopping],  # Adiciona o callback EarlyStopping
        verbose=0  # Mudar para 1 se quiser ver o progresso de treinamento
    )

    # Faz previsões
    predictions = GRU_model.predict(X_test)  # Gera previsões para os dados de teste

    # Desfaz a normalização para calcular o IoU
    predictions_unscaled = scaler.inverse_transform(predictions)  # Desfaz a normalização das previsões
    y_test_unscaled = scaler.inverse_transform(y_test)  # Desfaz a normalização dos valores reais de teste

    # Garante que test_metadata tenha o mesmo comprimento que as predições
    correct_length = len(y_test_unscaled)  # Obtém o comprimento correto dos dados de teste
    if len(test_metadata) > correct_length:
        test_metadata = test_metadata.iloc[:correct_length]  # Trunca o metadata para combinar com o comprimento das predições
    elif len(test_metadata) < correct_length:
        # Truncar as predições para corresponder ao metadata
        predictions_unscaled = predictions_unscaled[:len(test_metadata)]
        y_test_unscaled = y_test_unscaled[:len(test_metadata)]
        correct_length = len(test_metadata)  # Atualiza o comprimento correto

    # Calcula o IoU
    bboxes_real = y_test_unscaled  # shape: (num_samples, 4)
    bboxes_pred = predictions_unscaled  # shape: (num_samples, 4)
    ious = calcular_iou_vetorizado(bboxes_real, bboxes_pred)  # Calcula o IoU de forma vetorizada
    mean_iou = np.mean(ious)  # Calcula a média do IoU
    print(f'IoU Médio do modelo GRU: {mean_iou:.2f}')

    # Armazena o modelo e suas informações de configuração e desempenho
    best_models.append({
        'model': GRU_model,
        'iou': mean_iou,
        'config': {
            'sequence_length': sequence_length,
            'units': units,
            'dropout_rate': dropout_rate,
            'epochs': epochs,
            'activation_name': activation_name,
            'learning_rate': learning_rate
        },
        'scaler': scaler,
        'test_inputs': X_test,
        'test_outputs': y_test,
        'test_metadata': test_metadata
    })

# Ordena os modelos pelo maior IoU e mantém os 10 melhores
best_models = sorted(best_models, key=lambda x: x['iou'], reverse=True)[:10]  # Classifica e seleciona os 10 melhores modelos com base no IoU

# Imprime as 10 melhores configurações encontradas
for idx, best_model in enumerate(best_models, start=1):
    print(f'\nModelo {idx}:')
    print(f'Sequence Length: {best_model["config"]["sequence_length"]}')
    print(f'Units: {best_model["config"]["units"]}')
    print(f'Dropout Rate: {best_model["config"]["dropout_rate"]}')
    print(f'Epochs: {best_model["config"]["epochs"]}')
    print(f'Activation Name: {best_model["config"]["activation_name"]}')
    print(f'Learning Rate: {best_model["config"]["learning_rate"]}')
    print(f'IoU: {best_model["iou"]:.2f}')


# Cálculo de movimentos das bounding boxes para análise de deslocamento de objetos

In [ ]:
# Função para analisar movimentos das bounding boxes
def analyze_bbox_movements(df):
    # Ordena por instance_token, camera e timestamp para garantir a ordem correta
    df = df.sort_values(by=['instance_token', 'camera', 'timestamp']).reset_index(drop=True)  # Ordena o DataFrame para garantir a sequência correta dos dados
    distances = []  # Lista para armazenar as distâncias calculadas entre bounding boxes consecutivas

    unique_instances = df['instance_token'].unique()  # Obtém uma lista única de instâncias (objetos) no DataFrame

    for instance in unique_instances:
        instance_data = df[df['instance_token'] == instance].reset_index(drop=True)  # Filtra os dados para a instância atual e reseta o índice
        cameras = instance_data['camera'].unique()  # Obtém uma lista única de câmeras para a instância atual

        for cam in cameras:
            cam_data = instance_data[instance_data['camera'] == cam].reset_index(drop=True)  # Filtra os dados para a câmera atual e reseta o índice
            bboxes = cam_data[['bbox_min_x', 'bbox_min_y', 'bbox_max_x', 'bbox_max_y']].values  # Extrai as coordenadas das bounding boxes como array

            for i in range(1, len(bboxes)):
                bbox_prev = bboxes[i - 1]  # Bounding box anterior
                bbox_curr = bboxes[i]  # Bounding box atual

                # Calcula os centros das bounding boxes
                center_prev = [(bbox_prev[0] + bbox_prev[2]) / 2, (bbox_prev[1] + bbox_prev[3]) / 2]  # Centro da bounding box anterior
                center_curr = [(bbox_curr[0] + bbox_curr[2]) / 2, (bbox_curr[1] + bbox_curr[3]) / 2]  # Centro da bounding box atual

                # Calcula a distância euclidiana entre os centros
                dist = np.linalg.norm(np.array(center_curr) - np.array(center_prev))  # Calcula a distância entre os centros das bounding boxes
                distances.append(dist)  # Adiciona a distância à lista de distâncias

    return distances  # Retorna a lista de distâncias calculadas

# Calcula as distâncias no conjunto de treinamento
distances = analyze_bbox_movements(train_data)  # Calcula as distâncias entre movimentos das bounding boxes no conjunto de treino

# Converte as distâncias para um array NumPy
distances = np.array(distances)  # Converte a lista de distâncias para um array NumPy

# Calcula resumos estatísticos
mean_distance = np.mean(distances)  # Calcula a distância média
median_distance = np.median(distances)  # Calcula a mediana das distâncias
std_distance = np.std(distances)  # Calcula o desvio padrão das distâncias
min_distance = np.min(distances)  # Calcula a distância mínima
max_distance = np.max(distances)  # Calcula a distância máxima

# Imprime os resumos estatísticos
print("\nResumos Estatísticos dos Movimentos das Bounding Boxes:")
print(f"Distância Média: {mean_distance:.2f}")
print(f"Mediana da Distância: {median_distance:.2f}")
print(f"Desvio Padrão: {std_distance:.2f}")
print(f"Distância Mínima: {min_distance:.2f}")
print(f"Distância Máxima: {max_distance:.2f}")


# Algoritmo de rastreamento simples para identificação e acompanhamento de objetos por câmera

In [ ]:
# Função de Rastreamento Simples
def simple_tracker(pred_bboxes, timestamps, cameras, max_distance=median_distance):
    pred_ids_array = np.zeros(len(pred_bboxes), dtype=int)  # Inicializa o array de IDs das predições com zeros
    next_id = 1  # Define o próximo ID de track como 1
    active_tracks_per_camera = {}  # Dicionário para armazenar tracks ativos por câmera

    # Inicializa tracks ativos para cada câmera
    for cam in np.unique(cameras):
        active_tracks_per_camera[cam] = {}

    # Ordena índices para processar em ordem de timestamp e câmera
    sort_idx = np.lexsort((timestamps, cameras))  # Ordena índices usando câmera e timestamp
    pred_bboxes_sorted = pred_bboxes[sort_idx]  # Ordena bounding boxes de acordo com os índices
    timestamps_sorted = timestamps[sort_idx]  # Ordena timestamps de acordo com os índices
    cameras_sorted = cameras[sort_idx]  # Ordena câmeras de acordo com os índices

    for idx in range(len(pred_bboxes_sorted)):
        cam = cameras_sorted[idx]  # Obtém a câmera atual
        t = timestamps_sorted[idx]  # Obtém o timestamp atual
        bbox = pred_bboxes_sorted[idx]  # Obtém a bounding box atual
        idx_original = sort_idx[idx]  # Índice original da bounding box

        active_tracks = active_tracks_per_camera[cam]  # Obtém tracks ativos para a câmera atual

        # Se não houver tracks ativos, cria um novo
        if not active_tracks:
            pred_ids_array[idx_original] = next_id  # Atribui o próximo ID ao array de predições
            active_tracks[next_id] = {'bbox': bbox, 'timestamp': t}  # Salva o novo track com bounding box e timestamp
            next_id += 1  # Incrementa o próximo ID
        else:
            # Calcula distâncias para tracks ativos
            track_ids = list(active_tracks.keys())  # Obtém os IDs dos tracks ativos
            track_bboxes = np.array([active_tracks[tid]['bbox'] for tid in track_ids])  # Extrai as bounding boxes dos tracks ativos

            distances = cdist(track_bboxes, [bbox], metric='euclidean').flatten()  # Calcula distâncias euclidianas entre bounding boxes
            min_dist_idx = np.argmin(distances)  # Índice da menor distância
            min_dist = distances[min_dist_idx]  # Menor distância encontrada

            if min_dist < max_distance:
                tid = track_ids[min_dist_idx]  # ID do track com menor distância
                pred_ids_array[idx_original] = tid  # Atribui o ID existente ao array de predições
                active_tracks[tid] = {'bbox': bbox, 'timestamp': t}  # Atualiza o track ativo com a nova bounding box e timestamp
            else:
                pred_ids_array[idx_original] = next_id  # Atribui um novo ID ao array de predições
                active_tracks[next_id] = {'bbox': bbox, 'timestamp': t}  # Adiciona o novo track aos ativos
                next_id += 1  # Incrementa o próximo ID

    return pred_ids_array  # Retorna o array de IDs preditos



Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=swish, epochs=10
IoU Médio do modelo GRU: 0.15

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=swish, epochs=30
IoU Médio do modelo GRU: 0.26

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=swish, epochs=200
IoU Médio do modelo GRU: 0.42

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=relu, epochs=10
IoU Médio do modelo GRU: 0.11

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=relu, epochs=30
IoU Médio do modelo GRU: 0.27

Treinando com hiperparâmetros: sequence_length=4, units=256, dropout_rate=0.1, learning_rate=0.001, activation=relu, epochs=200
IoU Médio do modelo GRU: 0.26

Treinando com hiperparâmetros: sequence_length

# Aplicação das métricas MOTA, IDS, FN, FP, e demais métricas relevantes para análise dos resultados.

In [ ]:
# Função de aplicação de métricas
def apply_mot_metrics(df):
    # Cria um mapeamento único para cada combinação de instance_token e câmera
    df['gt_id'] = df.apply(lambda row: f"{row['instance_token']}_{row['camera']}", axis=1)

    # Mapeia cada combinação única para um ID numérico único
    unique_gt_ids = df['gt_id'].unique()
    id_mapping_gt = {token: idx + 1 for idx, token in enumerate(unique_gt_ids)}  # Inicia em 1
    df['gt_id_num'] = df['gt_id'].map(id_mapping_gt)

    # Inicializa o acumulador com auto_id=False
    acc = mm.MOTAccumulator(auto_id=False)

    # Atribui um frameid global sequencial para cada timestamp único
    unique_timestamps = sorted(df['timestamp'].unique())
    timestamp_to_frameid = {timestamp: idx for idx, timestamp in enumerate(unique_timestamps)}
    df['frameid'] = df['timestamp'].map(timestamp_to_frameid)

    # Ordena o DataFrame por frameid
    df = df.sort_values(by=['frameid'])

    # Agrupa as detecções por frameid
    grouped = df.groupby('frameid')

    for frameid, group in grouped:
        # IDs e bounding boxes de ground truth
        gt_ids = group['gt_id_num'].tolist()
        gt_bboxes = group[['true_min_x', 'true_min_y', 'true_max_x', 'true_max_y']].values

        # IDs e bounding boxes preditos
        pred_ids_frame = group['pred_id'].tolist()
        pred_bboxes_frame = group[['pred_min_x', 'pred_min_y', 'pred_max_x', 'pred_max_y']].values

        # Converte IDs para strings
        gt_ids = [str(id) for id in gt_ids]
        pred_ids_frame = [str(id) for id in pred_ids_frame]

        # Verifica se há duplicidades de gt_id no mesmo frameid
        duplicates = group.duplicated(subset=['gt_id_num'])
        if duplicates.any():
            print(f"Removendo {duplicates.sum()} duplicatas de gt_id no frameid {frameid}.")
            group = group.drop_duplicates(subset=['gt_id_num'], keep='first')
            gt_ids = group['gt_id_num'].tolist()
            gt_bboxes = group[['true_min_x', 'true_min_y', 'true_max_x', 'true_max_y']].values
            distances = mm.distances.iou_matrix(gt_bboxes, pred_bboxes_frame, max_iou=0.5)

        # Calcula a matriz de distâncias (usando IoU)
        distances = mm.distances.iou_matrix(gt_bboxes, pred_bboxes_frame, max_iou=0.5)

        # Atualiza o acumulador com as detecções deste frame
        acc.update(
            gt_ids,
            pred_ids_frame,
            distances,
            frameid=frameid
        )

    return acc  # Retorna o acumulador com os dados de rastreamento


metrics_results = []  # Lista para armazenar resultados das métricas

for idx, best_model in enumerate(best_models, start=1):
    print(f'\nProcessando Métricas para Modelo {idx}...')

    GRU_model = best_model['model']
    scaler = best_model['scaler']
    X_test = best_model['test_inputs']
    y_test = best_model['test_outputs']
    test_metadata = best_model['test_metadata']

    # Faz previsões com o modelo atual
    predictions = GRU_model.predict(X_test)

    # Desfaz a normalização para calcular métricas
    predictions_unscaled = scaler.inverse_transform(predictions)
    y_test_unscaled = scaler.inverse_transform(y_test)

    # Garante que test_metadata tenha o mesmo comprimento que as predições
    correct_length = len(y_test_unscaled)
    if len(test_metadata) > correct_length:
        test_metadata = test_metadata.iloc[:correct_length]
    elif len(test_metadata) < correct_length:
        # Trunca as predições para corresponder ao metadata
        predictions_unscaled = predictions_unscaled[:len(test_metadata)]
        y_test_unscaled = y_test_unscaled[:len(test_metadata)]
        correct_length = len(test_metadata)

    # Cria DataFrame para métricas
    df_results = pd.DataFrame({
        'instance_token': test_metadata['instance_token'],
        'timestamp': test_metadata['timestamp'],
        'camera': test_metadata['camera'],
        'scene_token': test_metadata['scene_token'],
        'true_min_x': y_test_unscaled[:, 0],
        'true_max_x': y_test_unscaled[:, 1],
        'true_min_y': y_test_unscaled[:, 2],
        'true_max_y': y_test_unscaled[:, 3],
        'pred_min_x': predictions_unscaled[:, 0],
        'pred_max_x': predictions_unscaled[:, 1],
        'pred_min_y': predictions_unscaled[:, 2],
        'pred_max_y': predictions_unscaled[:, 3]
    })

    # Adiciona colunas necessárias para o tracker
    df_results['timestamp'] = df_results['timestamp'].astype(int)
    pred_bboxes = df_results[['pred_min_x', 'pred_min_y', 'pred_max_x', 'pred_max_y']].values
    timestamps = df_results['timestamp'].values
    cameras = df_results['camera'].values

    # Aplica o tracker para atribuir IDs preditos
    pred_ids = simple_tracker(pred_bboxes, timestamps, cameras, max_distance=median_distance)
    df_results['pred_id'] = pred_ids

    # Aplica as métricas
    accumulator = apply_mot_metrics(df_results)

    # Cria um manipulador de métricas para gerar métricas dos dados acumulados
    mh = mm.metrics.create()

    # Calcula várias métricas
    summary = mh.compute(
        accumulator,
        metrics=['num_frames', 'num_switches', 'mota', 'motp', 'idf1', 'idp', 'idr', 'recall', 'precision'],
        name=f'Model_{idx}'
    )

    # Adiciona resultado ao conjunto de resultados
    metrics_results.append(summary)

    print(f'Métricas calculadas para Modelo {idx}.')

# Verifica se há resultados para serem concatenados e salva em um arquivo CSV
if metrics_results:
    metrics_summary_df = pd.concat(metrics_results, axis=0).reset_index(drop=True)
    metrics_summary_df.to_csv('mot_metrics_summary.csv', index=False)
    print('\nMétricas MOT salvas em mot_metrics_summary.csv')
else:
    print('Nenhuma métrica calculada para os modelos selecionados.')

# Imprime o resumo das métricas
print(metrics_summary_df)
